In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K

## Classification and Imputation using Auto-Encoder

The idea is to train a dense neural network as an auto-encoder to fill in missing values.


## Outputs
Creates output files in the "predictions" directory.
Imputes as much as possible all NaNs in both train and test files and adds predictions.

**IMPORTANT**: Preidictions in the train set were created by the Auto-Encoder!!


In [2]:
# Load data
train = pd.read_csv('data/trainSet.txt')
test = pd.read_csv('data/testSet.txt')

In [3]:
# definde columns of interest
cols_out = ['PatientID', 'ImageFile', 'Hospital', 'Prognosis']
binary_vars = ['RespiratoryFailure', 'Sex', 'Cough', 'DifficultyInBreathing', 'CardiovascularDisease']
impute_cols = [i for i in train.columns if i not in cols_out]

## Autoencoder should work!

In [31]:
def create_model():
    
    regularizer = keras.regularizers.l1_l2(l1=0, l2=0.005)
    
    inputs = keras.Input(shape=(16,))
    x = keras.layers.Dense(16, activation='selu')(inputs)
    # build a little auto-encoder
    x = keras.layers.Dense(80, activation='selu', kernel_regularizer=regularizer)(x)
    x = keras.layers.Dropout(0.5)(x)
    # bottleneck
    x = keras.layers.Dense(4, activation='selu', kernel_regularizer=regularizer)(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(80, activation='selu', kernel_regularizer=regularizer)(x)
    x = keras.layers.Dropout(0.5)(x)
    
    out1 = keras.layers.Dense(16, activation='sigmoid', name='imputing')(x)
    out2 = keras.layers.Dense(1, activation='sigmoid', name='classifying')(x)
    
    model = keras.Model(inputs, {'impute': out1, 'classify':out2})
    
    return model

In [32]:
# From https://github.com/keras-team/keras/issues/7065
# Calculate MSE only on known values
def masked_mse(mask_value=-1):
    def f(y_true, y_pred):
        mask_true = K.cast(K.not_equal(y_true, mask_value), K.floatx())
        masked_squared_error = K.square(mask_true * (y_true - y_pred))
        masked_mse = K.sum(masked_squared_error, axis=-1) / K.sum(mask_true, axis=-1)
        return masked_mse * 100

    return f


# Validate imputation by removing one nan extra per column
def imputer_validation(valid_set):
    """
    Function to remove extra fields from data for validation.
    """
    fields_removed = valid_set.copy()
    imputer_mask = np.zeros(valid_set.shape)
    for n, row in enumerate(fields_removed):
        non_nan = np.where(row != -1)[0]
        set_nan = np.random.choice(non_nan)
        fields_removed[n, set_nan] = -1
        imputer_mask[n, set_nan] = True
    
    return fields_removed, imputer_mask


# helper to calculate scores
def calculate_score(x_true, x_pred, errorfun='mse', scaler=None):
    if scaler:
        x_true = scaler.inverse_transform(x_true.copy())
        x_pred = scaler.inverse_transform(x_pred.copy())
        x_true = np.clip(x_true, a_min=-1, a_max=None)
        x_pred = np.clip(x_pred, a_min=-1, a_max=None)
    if errorfun == 'mse':
        score = mean_squared_error(x_true, x_pred)
    elif errorfun == 'acc':
        score = accuracy_score(x_true, x_pred > 0.5)
    elif errorfun == "masked_mse":
        score = masked_mse(-1)(x_true, x_pred)
        score = np.mean(score)
    
    return score


def post_processing(original_df, predicted_values, classify, scaler=None):
    # Post processing
    nan_mask = np.isnan(original_df)
    previous_values = original_df.copy().values
    imputed_thing = np.zeros(previous_values.shape)

    # Rescale values and add to empty matrix
    if scaler is not None:
        predicted_values = MM.inverse_transform(predicted_values)
    
    imputed_thing[nan_mask] = predicted_values[nan_mask]
    # Clip negatives
    imputed_thing = np.clip(imputed_thing, a_min=0, a_max=None) 
    # Add known values
    imputed_thing[nan_mask==False] = previous_values[nan_mask==False]
    
    # Add classification and column names, binarize binary values
    df_out = pd.DataFrame(imputed_thing, columns=original_df.columns)
    df_out[binary_vars] = (df_out[binary_vars] > 0.5) * 1.0
    df_out['Prognosis'] = classify > 0.5
    
    return df_out

In [33]:
# some global vars
early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
OPTIMIZER = keras.optimizers.Adam(lr=0.0005)
CALLBACKS = [early_stopping]
LOSS = {'impute': masked_mse(), 'classify': 'binary_crossentropy'}

In [34]:
# Preprare data for training
train_clean = train[impute_cols]
train_y = train['Prognosis'] == 'MILD'

In [35]:
accs, mses0, mses1, imps0, imps1, trues, preds, indices, hosp_val = [], [], [], [], [], [], [], [], []

train_mask = np.isnan(train_clean).values

for hosp in np.unique(train.Hospital):
    model = create_model()
    model.compile(optimizer=OPTIMIZER, 
                  loss=LOSS,
                  metrics={'impute': 'mse', 'classify': 'accuracy'})
    MM = MinMaxScaler(clip=True)
    
    tr = np.where(train.Hospital != hosp)[0]
    vl = np.where(train.Hospital == hosp)[0]

    # Remove Nans
    x_train = train_clean.iloc[tr].copy().fillna(0)
    x_valid = train_clean.iloc[vl].copy().fillna(0)
    # Define y-values
    y_train = train_y.iloc[tr]
    y_valid = train_y.iloc[vl]
    # Scale data from 0 to 1
    x_train = MM.fit_transform(x_train)
    x_valid = MM.transform(x_valid)
    
    # Set former nan values to -1
    x_train[train_mask[tr]] = -1
    x_valid[train_mask[vl]] = -1
    
    # Fit Model
    model.fit(x_train, {'impute':x_train, 'classify': y_train}, verbose=0, 
              callbacks=CALLBACKS, validation_split=0.15,
             epochs=250)
    
    # Predict the things
    prediction = model.predict(x_valid)
    preds.append(MM.inverse_transform(prediction['impute']))

    pred_val, pred_y = prediction['impute'], prediction['classify']
    
    
    # Returns metrics, weirdly
    _, _, _, train_acc, train_mse = model.evaluate(x_train, 
                                                   {'impute':x_train, 
                                                    'classify': y_train}, 
                                                   verbose=0)
    
    # Set previous NAN to -1
    pred_val[train_mask[vl]] = -1
    x_valid[train_mask[vl]] = -1
    
    # imputer validation:
    x_impute, impute_mask = imputer_validation(x_valid)
    
    imputer_predict = model.predict(x_impute)['impute']
    imputer_predict[impute_mask==False] = -1
    
    validation_mask = x_valid.copy()
    validation_mask[impute_mask==False] = -1
    
    # imputer error
    imp0 = calculate_score(validation_mask, 
                            imputer_predict, errorfun='masked_mse')
    imp1 = calculate_score(validation_mask, 
                            imputer_predict, errorfun='masked_mse', scaler=MM)
    
    # Keepingt track
    imps0.append(imp0)
    imps1.append(imp1)
    mses0.append(calculate_score(x_valid, pred_val, scaler=MM, errorfun='masked_mse'))
    mses1.append(calculate_score(x_valid, pred_val, errorfun='masked_mse'))
    accs.append(calculate_score(y_valid, pred_y, errorfun='acc'))
    trues.append(MM.inverse_transform(x_valid))
    preds.append(MM.inverse_transform(prediction['impute']))
    print(f"Split {hosp}, with validation n = {x_valid.shape[0]}: \nRescaled MSE - {mses0[-1]:4.2f} " +
          f"MSE - {mses1[-1]:4.2f} " +
          f"Acc - {accs[-1]:4.2f}\n" +
          f"Impute MSE - {imps0[-1]:4.2f} " +
          f"Impute MSE scaled - {imps1[-1]:4.2f}")
          
    print(f"Train - MSE {train_mse:4.3f}, ACC {train_acc:4.3f}")
    
    indices.append(vl)
    hosp_val.append([hosp] * len(vl))
    print("==============================================")

Split B, with validation n = 104: 
Rescaled MSE - 79392552.00 MSE - 1.53 Acc - 0.70
Impute MSE - 14.47 Impute MSE scaled - 462034272.00
Train - MSE 0.562, ACC 0.651
Split C, with validation n = 31: 
Rescaled MSE - 7313358.50 MSE - 2.03 Acc - 0.65
Impute MSE - 19.34 Impute MSE scaled - 128932.62
Train - MSE 0.521, ACC 0.633
Split D, with validation n = 139: 
Rescaled MSE - 116053528.00 MSE - 3.69 Acc - 0.65
Impute MSE - 16.61 Impute MSE scaled - 6011308.50
Train - MSE 0.556, ACC 0.635
Split E, with validation n = 101: 
Rescaled MSE - 166108.31 MSE - 1.07 Acc - 0.65
Impute MSE - 11.43 Impute MSE scaled - 400864.69
Train - MSE 0.526, ACC 0.655
Split F, with validation n = 488: 
Rescaled MSE - 93001744.00 MSE - 2.86 Acc - 0.61
Impute MSE - 11.17 Impute MSE scaled - 240221632.00
Train - MSE 0.491, ACC 0.680


# Final Training and Creating Test Set

In [36]:
model = create_model()
model.compile(optimizer=OPTIMIZER, 
                  loss=LOSS,
                  metrics={'impute': 'mae', 'classify': 'accuracy'})

MM = MinMaxScaler(clip=True)

# Transform Data:
x_train = train_clean.copy().fillna(0)
# Define y-values
y_train = train_y
# Scale data from 0 to 1
x_train = MM.fit_transform(x_train)
# Set former nan values to -1
x_train[train_mask] = -1


model.fit(x_train, {'impute':x_train, 'classify': y_train}, verbose=1, callbacks=CALLBACKS, 
          validation_split=0.15, epochs=250)

Epoch 1/250
23/23 [==============================] - 0s 14ms/step - loss: 16.1035 - classifying_loss: 0.8357 - imputing_loss: 15.0729 - classifying_accuracy: 0.4734 - imputing_mae: 0.5639 - val_loss: 10.2311 - val_classifying_loss: 0.7003 - val_imputing_loss: 9.3421 - val_classifying_accuracy: 0.4615 - val_imputing_mae: 0.5182
Epoch 2/250
23/23 [==============================] - 0s 3ms/step - loss: 11.8159 - classifying_loss: 0.7830 - imputing_loss: 10.8492 - classifying_accuracy: 0.4952 - imputing_mae: 0.5069 - val_loss: 9.3200 - val_classifying_loss: 0.6953 - val_imputing_loss: 8.4464 - val_classifying_accuracy: 0.4923 - val_imputing_mae: 0.5087
Epoch 3/250
23/23 [==============================] - 0s 3ms/step - loss: 10.7004 - classifying_loss: 0.7558 - imputing_loss: 9.7701 - classifying_accuracy: 0.4857 - imputing_mae: 0.4925 - val_loss: 8.9884 - val_classifying_loss: 0.6925 - val_imputing_loss: 8.1255 - val_classifying_accuracy: 0.5231 - val_imputing_mae: 0.5015
Epoch 4/250
23/23 

23/23 [==============================] - 0s 3ms/step - loss: 5.8571 - classifying_loss: 0.6831 - imputing_loss: 5.0371 - classifying_accuracy: 0.5675 - imputing_mae: 0.4245 - val_loss: 3.5190 - val_classifying_loss: 0.6511 - val_imputing_loss: 2.7314 - val_classifying_accuracy: 0.6462 - val_imputing_mae: 0.4390
Epoch 27/250
23/23 [==============================] - 0s 3ms/step - loss: 6.0063 - classifying_loss: 0.6829 - imputing_loss: 5.1872 - classifying_accuracy: 0.5553 - imputing_mae: 0.4250 - val_loss: 3.4490 - val_classifying_loss: 0.6521 - val_imputing_loss: 2.6611 - val_classifying_accuracy: 0.6462 - val_imputing_mae: 0.4380
Epoch 28/250
23/23 [==============================] - 0s 3ms/step - loss: 5.8844 - classifying_loss: 0.6791 - imputing_loss: 5.0697 - classifying_accuracy: 0.5853 - imputing_mae: 0.4225 - val_loss: 3.4108 - val_classifying_loss: 0.6534 - val_imputing_loss: 2.6223 - val_classifying_accuracy: 0.6462 - val_imputing_mae: 0.4377
Epoch 29/250
23/23 [===============

23/23 [==============================] - 0s 3ms/step - loss: 5.4691 - classifying_loss: 0.6631 - imputing_loss: 4.6833 - classifying_accuracy: 0.5812 - imputing_mae: 0.4155 - val_loss: 2.9223 - val_classifying_loss: 0.6532 - val_imputing_loss: 2.1465 - val_classifying_accuracy: 0.6692 - val_imputing_mae: 0.4284
Epoch 52/250
23/23 [==============================] - 0s 3ms/step - loss: 5.4862 - classifying_loss: 0.6796 - imputing_loss: 4.6842 - classifying_accuracy: 0.6139 - imputing_mae: 0.4161 - val_loss: 2.9410 - val_classifying_loss: 0.6555 - val_imputing_loss: 2.1633 - val_classifying_accuracy: 0.6692 - val_imputing_mae: 0.4285
Epoch 53/250
23/23 [==============================] - 0s 3ms/step - loss: 5.5625 - classifying_loss: 0.6791 - imputing_loss: 4.7613 - classifying_accuracy: 0.5730 - imputing_mae: 0.4180 - val_loss: 2.8692 - val_classifying_loss: 0.6516 - val_imputing_loss: 2.0956 - val_classifying_accuracy: 0.6615 - val_imputing_mae: 0.4282
Epoch 54/250
23/23 [===============

23/23 [==============================] - 0s 3ms/step - loss: 5.4185 - classifying_loss: 0.6695 - imputing_loss: 4.6363 - classifying_accuracy: 0.5975 - imputing_mae: 0.4129 - val_loss: 2.6771 - val_classifying_loss: 0.6487 - val_imputing_loss: 1.9158 - val_classifying_accuracy: 0.6615 - val_imputing_mae: 0.4237
Epoch 77/250
23/23 [==============================] - 0s 3ms/step - loss: 5.1934 - classifying_loss: 0.6504 - imputing_loss: 4.4305 - classifying_accuracy: 0.6262 - imputing_mae: 0.4113 - val_loss: 2.6674 - val_classifying_loss: 0.6492 - val_imputing_loss: 1.9058 - val_classifying_accuracy: 0.6615 - val_imputing_mae: 0.4230
Epoch 78/250
23/23 [==============================] - 0s 3ms/step - loss: 5.5025 - classifying_loss: 0.6905 - imputing_loss: 4.6999 - classifying_accuracy: 0.5880 - imputing_mae: 0.4137 - val_loss: 2.6969 - val_classifying_loss: 0.6499 - val_imputing_loss: 1.9350 - val_classifying_accuracy: 0.6615 - val_imputing_mae: 0.4229
Epoch 79/250
23/23 [===============

23/23 [==============================] - 0s 3ms/step - loss: 5.4225 - classifying_loss: 0.6580 - imputing_loss: 4.6594 - classifying_accuracy: 0.6098 - imputing_mae: 0.4114 - val_loss: 2.4990 - val_classifying_loss: 0.6552 - val_imputing_loss: 1.7389 - val_classifying_accuracy: 0.6692 - val_imputing_mae: 0.4196
Epoch 102/250
23/23 [==============================] - 0s 3ms/step - loss: 5.5162 - classifying_loss: 0.6740 - imputing_loss: 4.7375 - classifying_accuracy: 0.5798 - imputing_mae: 0.4133 - val_loss: 2.5005 - val_classifying_loss: 0.6531 - val_imputing_loss: 1.7429 - val_classifying_accuracy: 0.6692 - val_imputing_mae: 0.4198
Epoch 103/250
23/23 [==============================] - 0s 3ms/step - loss: 5.3019 - classifying_loss: 0.6681 - imputing_loss: 4.5295 - classifying_accuracy: 0.5812 - imputing_mae: 0.4118 - val_loss: 2.5507 - val_classifying_loss: 0.6513 - val_imputing_loss: 1.7952 - val_classifying_accuracy: 0.6615 - val_imputing_mae: 0.4199
Epoch 104/250
23/23 [============

23/23 [==============================] - 0s 3ms/step - loss: 5.2295 - classifying_loss: 0.6714 - imputing_loss: 4.4586 - classifying_accuracy: 0.5812 - imputing_mae: 0.4090 - val_loss: 2.4125 - val_classifying_loss: 0.6560 - val_imputing_loss: 1.6574 - val_classifying_accuracy: 0.6692 - val_imputing_mae: 0.4169
Epoch 127/250
23/23 [==============================] - 0s 3ms/step - loss: 5.3308 - classifying_loss: 0.6645 - imputing_loss: 4.5672 - classifying_accuracy: 0.5962 - imputing_mae: 0.4104 - val_loss: 2.4131 - val_classifying_loss: 0.6557 - val_imputing_loss: 1.6585 - val_classifying_accuracy: 0.6769 - val_imputing_mae: 0.4168
Epoch 128/250
23/23 [==============================] - 0s 3ms/step - loss: 5.2212 - classifying_loss: 0.6685 - imputing_loss: 4.4540 - classifying_accuracy: 0.5948 - imputing_mae: 0.4088 - val_loss: 2.4143 - val_classifying_loss: 0.6556 - val_imputing_loss: 1.6599 - val_classifying_accuracy: 0.6769 - val_imputing_mae: 0.4168
Epoch 129/250
23/23 [============

In [37]:
train_predictions = model.predict(x_train)
train_imputations = train_predictions['impute']
train_classify = train_predictions['classify']

train_out = post_processing(train_clean, train_imputations, train_classify, MM)

In [38]:
# Add missing columns
train_out[['PatientID', 'ImageFile', 'Hospital']] = train[['PatientID', 'ImageFile', 'Hospital']]
# Change order to original
train_out = train_out[train.columns]

train_out.to_csv('simon_TRAIN_impute_classify.csv', index=None)

In [39]:
test_clean = test[impute_cols] 
x_test = test_clean.copy().fillna(0)
x_test = MM.transform(x_test)
x_test[np.isnan(test_clean)] = -1

In [40]:
predictions = model.predict(x_test)

In [41]:
# Predidctions
imputations = predictions['impute']
classify = predictions['classify']

In [42]:
test_out = post_processing(test_clean, imputations, classify, MM)

test_out[test_out.Prognosis==True] = "MILD"
test_out[test_out.Prognosis==False] = "SEVERE"

In [43]:
# Add missing columns
test_out[['PatientID', 'ImageFile', 'Hospital']] = test[['PatientID', 'ImageFile', 'Hospital']]
# Change order to original
test_out = test_out[test.columns]

In [44]:
# Save to file
test_out.to_csv('predictions/simon_TEST_impute_classify.csv', index=False)